In [1]:
#!/usr/bin/env python3
import sys
import smach
import rospy
import cv2 as cv
import numpy as np
from std_srvs.srv import Empty
import moveit_commander
import moveit_msgs.msg
from geometry_msgs.msg import PoseStamped
import tf2_ros as tf2
from tf2_sensor_msgs.tf2_sensor_msgs import do_transform_cloud
from utils_takeshi import *
from grasp_utils import *

In [2]:
    global head, wb, arm, tf_man, gaze, robot, scene, calibrate_wrist #wbw, wbl
    global rgbd, hand_cam, wrist, gripper, grasp_base, clear_octo_client, service_client, AR_starter, AR_stopper

    moveit_commander.roscpp_initialize(sys.argv)
    rospy.init_node('pruebas_de')

    head = moveit_commander.MoveGroupCommander('head')
    wb = moveit_commander.MoveGroupCommander('whole_body')
    arm =  moveit_commander.MoveGroupCommander('arm')
    # wbl = moveit_commander.MoveGroupCommander('whole_body_light')
    #wbw.set_workspace([-6.0, -6.0, 6.0, 6.0]) 
    #wbl.set_workspace([-6.0, -6.0, 6.0, 6.0])  
    wb.set_workspace([-6.0, -6.0, 6.0, 6.0])  
    
    robot = moveit_commander.RobotCommander()
    scene = moveit_commander.PlanningSceneInterface()
    
#     tf_man = TF_MANAGER()
#     rgbd = RGBD()
#     hand_cam = HAND_RGB()
#     wrist = WRIST_SENSOR()
#     gripper = GRIPPER()
#     grasp_base = OMNIBASE()
#     gaze = GAZE()

    clear_octo_client = rospy.ServiceProxy('/clear_octomap', Empty)
    calibrate_wrist = rospy.ServiceProxy('/hsrb/wrist_wrench/readjust_offset',Empty)
    AR_starter = rospy.ServiceProxy('/marker/start_recognition',Empty)
    AR_stopper = rospy.ServiceProxy('/marker/stop_recognition',Empty)
    
    head.set_planning_time(0.3)
    head.set_num_planning_attempts(1)
    wb.set_num_planning_attempts(10)

[ WARN] [1671124070.176468667]: Link hand_l_finger_vacuum_frame has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying collision geometry.
[ WARN] [1671124070.180208918]: Link head_l_stereo_camera_link has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying collision geometry.
[ WARN] [1671124070.180320187]: Link head_r_stereo_camera_link has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying collision geometry.
[ WARN] [1671124070.181298750]: Group state 'neutral' doesn't specify all group joints in group 'arm'. wrist_ft_sensor_frame_joint is missing.
[ WARN] [1671124070.181339445]: Group state 'go' doesn't specify all group joints in group 'arm'. wrist_ft_sensor_frame_joint is missing.
[ WARN] [1671124086.963337732]: Kinematics solver doesn't support #attempts any

In [4]:
    
    tf_man = TF_MANAGER()
#     rgbd = RGBD()
#     hand_cam = HAND_RGB()
    wrist = WRIST_SENSOR()
    gripper = GRIPPER()
    grasp_base = OMNIBASE()
    gaze = GAZE()

In [151]:
AR_starter.call()
head.set_named_target('neutral')
head.go()
arm.set_named_target('neutral')
arm.go()

True

In [152]:
pos, rot = tf_man.getTF(target_frame='ar_marker/7')
tf_man.pub_static_tf(point_name='player', pos=pos, rot=rot)

In [153]:
# acp = arm.get_current_joint_values()
# print(acp)
arm.set_named_target('neutral')
arm.go()

True

In [154]:
acp = arm.get_current_joint_values()
acp[4] = 1.60807
arm.set_joint_value_target(acp)
arm.go()

True

In [133]:
        #Align with player AR 0
        succ = False
        THRESHOLD = 0.01
        while not succ:
            trans,_ = tf_man.getTF(target_frame='player', ref_frame='hand_palm_link')
            if type(trans) is not bool:
                eY, _, eX = trans
                eY = -eY 
                eX -= 0.25
                rospy.loginfo("Distance to goal: {:.3f}, {:.3f}".format(eX, eY))
                if abs(eY) < THRESHOLD:
                    eY = 0
                if abs(eX) < THRESHOLD:
                    eX = 0
                succ =  eX == 0 and eY == 0
#                 rospy.sleep(1.0)
                grasp_base.tiny_move(velX=0.2
                                     *eX, velY=0.4*eY, std_time=0.2, MAX_VEL=0.3) #Pending test

[INFO] [1671127703.527544]: Distance to goal: 0.040, 0.131
[INFO] [1671127703.773445]: Distance to goal: 0.040, 0.131
[INFO] [1671127704.043683]: Distance to goal: 0.044, 0.103
[INFO] [1671127704.295141]: Distance to goal: 0.046, 0.082
[INFO] [1671127704.568599]: Distance to goal: 0.044, 0.057
[INFO] [1671127704.808093]: Distance to goal: 0.044, 0.042
[INFO] [1671127705.038515]: Distance to goal: 0.040, 0.035
[INFO] [1671127705.271799]: Distance to goal: 0.041, 0.030
[INFO] [1671127705.510016]: Distance to goal: 0.041, 0.030
[INFO] [1671127705.768771]: Distance to goal: 0.037, 0.025
[INFO] [1671127706.009435]: Distance to goal: 0.036, 0.020
[INFO] [1671127706.242590]: Distance to goal: 0.033, 0.015
[INFO] [1671127706.464681]: Distance to goal: 0.033, 0.013
[INFO] [1671127706.682441]: Distance to goal: 0.027, 0.014
[INFO] [1671127706.896946]: Distance to goal: 0.030, 0.009
[INFO] [1671127707.135371]: Distance to goal: 0.027, 0.010
[INFO] [1671127707.354450]: Distance to goal: 0.021, 0.0

In [155]:
#Pre-place pose
pos, rot = tf_man.getTF(target_frame='hand_palm_link', ref_frame='odom')
trans,_ = tf_man.getTF(target_frame='player', ref_frame='hand_palm_link')
pos[2]+=(-trans[1]+0.0145)
tf_man.pub_static_tf(point_name='goal',pos=pos, rot=rot, ref='odom')
tf_man.pub_static_tf(point_name='goal_pose', pos=[0,0,0.04], ref='goal')
pos, rot = tf_man.getTF(target_frame='goal_pose', ref_frame='odom')
pose_goal = set_pose_goal(pos=pos, rot = rot)
arm.set_start_state_to_current_state()
arm.set_pose_target(pose_goal)
arm.go()

True

In [142]:
        #Align with player AR 1
        succ = False
        THRESHOLD = 0.008
        while not succ:
            trans,_ = tf_man.getTF(target_frame='player', ref_frame='hand_palm_link')
            if type(trans) is not bool:
                eY, _, eX = trans
                eY = -eY - 0.12
                eX -= 0.17
                rospy.loginfo("Distance to goal: {:.3f}, {:.3f}".format(eX, eY))
                if abs(eY) < THRESHOLD:
                    eY = 0
                if abs(eX) < THRESHOLD:
                    eX = 0
                succ =  eX == 0 and eY == 0
#                 rospy.sleep(1.0)
                grasp_base.tiny_move(velX=0.2
                                     *eX, velY=0.4*eY, std_time=0.2, MAX_VEL=0.3) #Pending test

[INFO] [1671127781.676516]: Distance to goal: -0.052, -0.107
[INFO] [1671127781.897611]: Distance to goal: -0.054, -0.106
[INFO] [1671127782.122805]: Distance to goal: -0.051, -0.095
[INFO] [1671127782.355446]: Distance to goal: -0.047, -0.086
[INFO] [1671127782.607502]: Distance to goal: -0.043, -0.077
[INFO] [1671127782.846435]: Distance to goal: -0.041, -0.069
[INFO] [1671127783.090094]: Distance to goal: -0.038, -0.063
[INFO] [1671127783.313760]: Distance to goal: -0.037, -0.059
[INFO] [1671127783.551085]: Distance to goal: -0.036, -0.058
[INFO] [1671127783.789197]: Distance to goal: -0.034, -0.049
[INFO] [1671127784.040034]: Distance to goal: -0.033, -0.044
[INFO] [1671127784.276884]: Distance to goal: -0.033, -0.042
[INFO] [1671127784.530825]: Distance to goal: -0.029, -0.037
[INFO] [1671127784.768280]: Distance to goal: -0.028, -0.031
[INFO] [1671127785.008439]: Distance to goal: -0.028, -0.025
[INFO] [1671127785.253849]: Distance to goal: -0.026, -0.022
[INFO] [1671127785.48068

In [118]:
        succ = False
        THRESHOLD = 0.08
        while not succ:
            try:
                trans, rot = tf_man.getTF(target_frame='player', ref_frame='base_link')
                euler = tf.transformations.euler_from_quaternion(rot)
                theta = euler[2]
                e = theta + 1.57
                rospy.loginfo("Its missing to turn {:.2f} radians".format(e))
                if abs(e) < THRESHOLD:
                    talk("I am aligned")
                    succ = True
#                     return 'succ'
                else:
                    rospy.sleep(0.1)
                    grasp_base.tiny_move(velT = 0.4*e, std_time=0.1)
            except:
                succ = True
#                 return 'tries'

[INFO] [1671127323.855274]: Its missing to turn -0.01 radians


In [141]:
        #Get player edge
        succ = False
        while not succ:
            force = wrist.get_force()
            if force[2] <= 0.0:
                grasp_base.tiny_move(velX=0.04, std_time=0.01)
            else:
                vel = -0.01
                grasp_base.tiny_move(velX=-0.01, std_time=0.01)
                p,r = tf_man.getTF(target_frame = 'hand_r_finger_tip_frame')
                tf_man.pub_static_tf(point_name='goal_pose',pos=p, rot=r, ref='map')
                succ = True

In [139]:
clear_octo_client()

In [54]:
calibrate_wrist = rospy.ServiceProxy('/hsrb/wrist_wrench/readjust_offset',Empty)

In [143]:
calibrate_wrist.call()

In [156]:
#Cuarta aproximacion (campos potenciales)
succ = False
e1= 1.0
n1 = 0.025 #Amplificacion de la fuerza lateral
n2 = 0.001      #Amplificacion de la fuerza frontal
c = 1.05        #
THRESHOLD = 0.001
while not succ:
    d, _ = tf_man.getTF(target_frame='player', ref_frame='hand_r_finger_tip_frame')
    d_GP,_ = tf_man.getTF(target_frame='goal_pose', ref_frame='hand_r_finger_tip_frame')
    force = wrist.get_force()
    dist = np.array([d[1],0.0])
    print(dist)
    if d_GP[1] >= 0.0:
        n1 = 0.025
    else:
        n1 = 0.001
    f = np.array([n2*force[2],-n1*force[0]])
    speed = e1*c*dist-f
    #print(f'Distance to AR x:{d[0]}, z:{d[1]}')
    succ =  (abs(dist[0]) < THRESHOLD)
    if not succ:
        grasp_base.tiny_move(velX = speed[0], velY = speed[1], std_time = 0.02, MAX_VEL = 0.01)
    print(dist[0])
# print(force)


[0.10497742 0.        ]
0.10497741988019005
[0.10497742 0.        ]
0.10497741988019005
[0.10497742 0.        ]
0.10497741988019005
[0.10427352 0.        ]
0.10427352257369416
[0.10713863 0.        ]
0.10713863180100225
[0.10725596 0.        ]
0.10725596034218343
[0.10725596 0.        ]
0.10725596034218343
[0.10183874 0.        ]
0.101838736789434
[0.10183874 0.        ]
0.101838736789434
[0.1031627 0.       ]
0.10316270039857578
[0.10699596 0.        ]
0.10699595907686177
[0.10539792 0.        ]
0.1053979199898214
[0.10529385 0.        ]
0.10529385391353907
[0.10587083 0.        ]
0.10587082936913983
[0.10603206 0.        ]
0.10603205819420669
[0.10273636 0.        ]
0.10273635871660436
[0.10247901 0.        ]
0.10247900592937231
[0.10247901 0.        ]
0.10247900592937231
[0.0986391 0.       ]
0.09863909687973293
[0.09954222 0.        ]
0.09954221897840121
[0.09977024 0.        ]
0.09977024309999782
[0.0997991 0.       ]
0.09979909903719231
[0.09816226 0.        ]
0.09816225545422375

0.07496242243630247
[0.07665138 0.        ]
0.07665137819464662
[0.07652655 0.        ]
0.07652655026728672
[0.07619409 0.        ]
0.07619409363613294
[0.07619409 0.        ]
0.07619409363613294
[0.07513091 0.        ]
0.07513091082152723
[0.07513091 0.        ]
0.07513091082152723
[0.07420626 0.        ]
0.07420626036542877
[0.07420626 0.        ]
0.07420626036542877
[0.07261684 0.        ]
0.07261683566421528
[0.07267684 0.        ]
0.07267684097573779
[0.07267684 0.        ]
0.07267684097573779
[0.07410784 0.        ]
0.0741078438376267
[0.07410784 0.        ]
0.0741078438376267
[0.07410784 0.        ]
0.0741078438376267
[0.07294743 0.        ]
0.07294743346505306
[0.07279915 0.        ]
0.07279915381613067
[0.07157326 0.        ]
0.07157326367717065
[0.07086151 0.        ]
0.07086151321840806
[0.07163382 0.        ]
0.07163381649303435
[0.07062544 0.        ]
0.07062544491199518
[0.07062544 0.        ]
0.07062544491199518
[0.06971137 0.        ]
0.06971136855924276
[0.06971137 0. 

0.03953377749582199
[0.03592455 0.        ]
0.03592455411972062
[0.03592455 0.        ]
0.03592455411972062
[0.04084524 0.        ]
0.04084524381843346
[0.04084524 0.        ]
0.04084524381843346
[0.04084524 0.        ]
0.04084524381843346
[0.03898644 0.        ]
0.038986435922822804
[0.03617821 0.        ]
0.03617820799688176
[0.03617821 0.        ]
0.03617820799688176
[0.03319542 0.        ]
0.03319541781045643
[0.03319542 0.        ]
0.03319541781045643
[0.03681915 0.        ]
0.036819154494170725
[0.03081121 0.        ]
0.030811209255741434
[0.02979342 0.        ]
0.029793416963863173
[0.02627616 0.        ]
0.026276155575065907
[0.02403839 0.        ]
0.024038385281116903
[0.02252858 0.        ]
0.022528575370953985
[0.02210779 0.        ]
0.022107786340254676
[0.03040718 0.        ]
0.030407177010707387
[0.03040718 0.        ]
0.030407177010707387
[0.03040854 0.        ]
0.030408539717344585
[0.02431836 0.        ]
0.02431835909871871
[0.02431836 0.        ]
0.02431835909871871
[

0.013357134905103818
[0.01335713 0.        ]
0.013357134905103818
[0.01455531 0.        ]
0.014555314281127107
[0.01342528 0.        ]
0.01342527643018121
[0.01342528 0.        ]
0.01342527643018121
[0.01101947 0.        ]
0.011019468571242697
[0.0127232 0.       ]
0.01272320068611954
[0.0127232 0.       ]
0.01272320068611954
[0.01342863 0.        ]
0.01342862681784518
[0.01212481 0.        ]
0.012124813324776773
[0.00677878 0.        ]
0.006778777547849391
[0.00612622 0.        ]
0.006126222261578551
[0.00612622 0.        ]
0.006126222261578551
[0.01126065 0.        ]
0.011260647789508527
[0.00854171 0.        ]
0.008541707625895567
[0.00822087 0.        ]
0.008220870989974705
[0.00899803 0.        ]
0.008998025720779346
[0.00901479 0.        ]
0.009014786897332372
[0.0064185 0.       ]
0.006418503540702281
[0.0064185 0.       ]
0.006418503540702281
[0.00672805 0.        ]
0.0067280480651352725
[0.00672805 0.        ]
0.0067280480651352725
[0.00991167 0.        ]
0.009911669528179456


0.01489032008496527
[0.01007522 0.        ]
0.010075221858870553
[0.0084445 0.       ]
0.008444499155689789
[0.0084445 0.       ]
0.008444499155689789
[0.00883986 0.        ]
0.008839864557916521
[0.01148691 0.        ]
0.011486907983389916
[0.01148691 0.        ]
0.011486907983389916
[0.01304564 0.        ]
0.013045643840769783
[0.01304564 0.        ]
0.013045643840769783
[0.01061578 0.        ]
0.010615775302520891
[0.00945997 0.        ]
0.009459971307671844
[0.00945997 0.        ]
0.009459971307671844
[0.00765329 0.        ]
0.007653292230099251
[0.00994807 0.        ]
0.009948071838033923
[0.00994807 0.        ]
0.009948071838033923
[0.00994807 0.        ]
0.009948071838033923
[0.01080224 0.        ]
0.010802242417169317
[0.01080224 0.        ]
0.010802242417169317
[0.01080224 0.        ]
0.010802242417169317
[0.01105517 0.        ]
0.011055167163258606
[0.01083882 0.        ]
0.010838822215870292
[0.0105776 0.       ]
0.010577601142676252
[0.01095971 0.        ]
0.010959705923838

0.011968835757362628
[0.01045647 0.        ]
0.01045647384793491
[0.01045647 0.        ]
0.01045647384793491
[0.01045647 0.        ]
0.01045647384793491
[0.01045647 0.        ]
0.01045647384793491
[0.01045647 0.        ]
0.01045647384793491
[0.01045647 0.        ]
0.01045647384793491
[0.01045647 0.        ]
0.01045647384793491
[0.01045647 0.        ]
0.01045647384793491
[0.01045647 0.        ]
0.01045647384793491
[0.01045647 0.        ]
0.01045647384793491
[0.01045647 0.        ]
0.01045647384793491
[0.01045647 0.        ]
0.01045647384793491
[0.01045647 0.        ]
0.01045647384793491
[0.00682469 0.        ]
0.006824688239383314
[0.00682469 0.        ]
0.006824688239383314
[0.00682469 0.        ]
0.006824688239383314
[0.00838133 0.        ]
0.008381331399713532
[0.00838133 0.        ]
0.008381331399713532
[0.00838133 0.        ]
0.008381331399713532
[0.00838133 0.        ]
0.008381331399713532
[0.00838133 0.        ]
0.008381331399713532
[0.00838133 0.        ]
0.008381331399713532
[0

KeyboardInterrupt: 

In [410]:
tf_man.getTF(target_frame='player', ref_frame='hand_r_finger_tip_frame')

[[-0.07518328202330665, -1.0368899942214682e-05, -0.0019421387175253546],
 [0.7297434939976142,
  -0.021727231492183204,
  0.02610371861785342,
  -0.682876970071593]]

In [157]:
gripper.steady()

In [158]:
succ = False
THRESHOLD = 0.07
while not succ:
    d, _ = tf_man.getTF(target_frame='player', ref_frame='hand_r_finger_tip_frame')
    succ = d[1] > THRESHOLD
    if not succ:
        grasp_base.tiny_move(velX = -0.04,std_time=0.2)

In [159]:
gripper.close()

In [160]:
succ = False
THRESHOLD = 0.04
while not succ:
    d, _ = tf_man.getTF(target_frame='player', ref_frame='hand_r_finger_tip_frame')
    succ = d[1] < THRESHOLD
    if not succ:
        grasp_base.tiny_move(velX = 0.04,std_time=0.2)

In [161]:
gripper.open()

In [97]:
arm.get_current_joint_values()

[0.12156869740565593,
 -0.11611952801888537,
 0.00042565465630595156,
 -1.4533416302849658,
 1.608075108223816,
 0.0]